<a href="https://colab.research.google.com/github/urosgodnov/BigData/blob/master/textMining_scraping_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Permanently install libraries

## Initializing gdrive

In [1]:
# permanently install packageg in google colab
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Creating links

In [5]:
import os
nb_path = '/content/drive/MyDrive/Google_Colab_modules'

if not os.path.exists(nb_path):
  os.symlink('/content/drive/MyDrive/Google_Colab_modules', nb_path)  # Create the symlink only if it doesn't exist
else:
  print(f"Path '{nb_path}' already exists. Skipping symlink creation.")


Path '/content/drive/MyDrive/Google_Colab_modules' already exists. Skipping symlink creation.


## Installing to destination folder in gdrive

In [ ]:
#install the module in the
!pip install --target=$nb_path selenium

In [ ]:
!pip install --target=$nb_path webdriver_manager

## Connecting the path

In the future, while all modules will be installed, we will only use this command.

In [25]:
import sys

sys.path.append("/content/drive/MyDrive/Google_Colab_modules")

# Webscraping

In [194]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    ElementClickInterceptedException,
    TimeoutException
)
from datetime import datetime



In [195]:
import pandas as pd
import time

In [196]:
options = webdriver.ChromeOptions()
options.add_argument("--verbose")
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument("--window-size=1920, 1200")
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like \
Gecko) Chrome/90.0.4430.212 Safari/537.36')

In [227]:
driver = webdriver.Chrome(options=options)

In [235]:
l="https://www.imdb.com/title/tt13622970/reviews/?ref_=tt_urv_sm"
driver.get(l)

In [236]:
wait = WebDriverWait(driver, timeout=35)


In [240]:
while True:
    try:
        # Locate the "25 More" button
        button = wait.until(EC.presence_of_element_located((By.XPATH, "//span[@class='ipc-see-more__text']")))
        print('Found the "25 More" button.')

        # Click the button using JavaScript
        driver.execute_script("arguments[0].click();", button)
        print('Clicked the "25 More" button.')

        time.sleep(3)


    except NoSuchElementException:
        print('The "25 More" button no longer exists. Exiting loop.')
        break  # Exit the loop when the button is no longer found

    except Exception as e:
        print('An unexpected error occurred:', e)
        break  # Exit the loop for other unexpected errors




Found the "25 More" button.
Clicked the "25 More" button.
Page loaded successfully.
Found the "25 More" button.
Clicked the "25 More" button.
Page loaded successfully.
Found the "25 More" button.
Clicked the "25 More" button.
Page loaded successfully.
Found the "25 More" button.
Clicked the "25 More" button.
Page loaded successfully.
An unexpected error occurred: Message: 
Stacktrace:
#0 0x59ec80e1b34a <unknown>
#1 0x59ec809316e0 <unknown>
#2 0x59ec809803e6 <unknown>
#3 0x59ec80980681 <unknown>
#4 0x59ec809c5b04 <unknown>
#5 0x59ec809a448d <unknown>
#6 0x59ec809c2ed7 <unknown>
#7 0x59ec809a4203 <unknown>
#8 0x59ec80972cc0 <unknown>
#9 0x59ec80973c9e <unknown>
#10 0x59ec80de8d3b <unknown>
#11 0x59ec80deccc2 <unknown>
#12 0x59ec80dd5b6c <unknown>
#13 0x59ec80ded837 <unknown>
#14 0x59ec80dbb10f <unknown>
#15 0x59ec80e0a5a8 <unknown>
#16 0x59ec80e0a770 <unknown>
#17 0x59ec80e1a1c6 <unknown>
#18 0x78e8cea42ac3 <unknown>



In [241]:
from bs4 import BeautifulSoup
content = driver.page_source
soup = BeautifulSoup(content)
result=[]
df = pd.DataFrame(columns=["Author","Title","Rating","Text","Date"])

In [189]:
def GetAReview(review):
  author = rev.findAll('a', attrs={'class':'ipc-link ipc-link--base'})[0].text
  title = rev.find('h3', attrs={'class':'ipc-title__text'}).text.strip()
  rating =rev.findAll('span', attrs={'class':'ipc-rating-star--rating'})[0].text.replace("\n","").strip()
  review = rev.findAll('div', attrs={'class':'ipc-html-content-inner-div'})[0].text
  date = rev.findAll('li', attrs={'class':'ipc-inline-list__item review-date'})[0].text
  date=datetime.strptime(date, '%b %d, %Y').strftime('%Y-%m-%d')

  return pd.DataFrame({"Author":author, "Title":title, "Rating":rating, "Text":review, "Date":date},index=[0])


In [242]:
for rev in soup.findAll('article'):
  try:
    result.append(GetAReview(rev))
  except:
    continue

In [243]:
df=pd.concat(result, axis=0, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Author  111 non-null    object
 1   Title   111 non-null    object
 2   Rating  111 non-null    object
 3   Text    111 non-null    object
 4   Date    111 non-null    object
dtypes: object(5)
memory usage: 4.5+ KB


In [ ]:
df.head()

In [ ]:
df.to_csv("Elevation.csv")